# Objects

Objects are a bunch of data with some attitude. Individuals like you and me are objects. We are data: our name, age, gender, credit score, income, traffic tickets, social security number, and dark secrets like a preference for pineapple on pizza.

Our behavior is our attitude. It determines how we share the data we are, with others. We readily answer the question "what's your name" but we think twice before revealing our social security number.

As objects, we use *judgement* to determine what information about ourselves to share freely with others and what to protect a bit. Ultimately, all information about ourselves in shareable under proper authority. For example, you may not disclose income and SSN to a stranger in the street. But you'll do so with a stranger behind the loan desk of a bank when it's time to buy a house.

Objects can be also programmed. An object-program does not have judgement, so it can't really tell for itself what information to share and what not. Left on its own, an object will share *everything* about itself. 

To instill a semblance of judgement in object oriented programming, we use *encapsulation.* This is a technique to instruct an object what data to share and how to share them.